# Dev procedure

In [ ]:
!pip install mediapipe opencv-python

In [3]:
# import libraries
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
# test camera
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('mp feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('0'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity = 0) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,)

        # print(results)

        cv2.imshow('mp feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
text = results.pose_landmarks
print(text)

In [6]:
# test different model
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

def test(com):
    prev_frame_time = 0
    new_frame_time = 0
    frame_cnt = 0
    total_fps = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity = com) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            new_frame_time = time.time()
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

            # print(results)
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time
            cv2.putText(image, 'FPS: {:.2f}'.format(fps), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

            cv2.imshow('mp feed', image)
            frame_cnt += 1
            total_fps += fps

            if cv2.waitKey(10) & 0xFF == ord('q'):
                print("Average FPS: ", total_fps/frame_cnt)
                break
test(0)
test(1)
test(2)
cap.release()
cv2.destroyAllWindows()

Average FPS:  29.126242943905403
Average FPS:  21.117740554167487
Average FPS:  12.414094768692417


# function implementation

In [7]:
import numpy as np
import cv2
import mediapipe as mp

In [23]:
def calcAngle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # usage 1
    # BA = a - b
    # BC = c - b

    # angle = np.degrees(np.arccos(np.dot((BA,BC) / (np.linalg.norm(BC) * np.linalg.norm(BC)))))
    # if angle > 180.0:
    #     angle = 360 - angle
    # return angle

    # usage 2
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [ ]:
# test show image
cv2.startWindowThread()
image = cv2.imread('testImage.jpg')
cv2.imshow('test', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
# image tester
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cv2.startWindowThread()

image = cv2.imread('testImage.jpg')

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity = 1)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image.flags.writeable = False

results = pose.process(image)

image.flags.writeable = True
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
)

landmarks = results.pose_landmarks.landmark

Lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
Lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
Lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

angle = calcAngle(Lshoulder, Lelbow, Lwrist)

cv2.putText(image, 'left hand' + str(int(angle)),
            tuple(np.multiply(Lelbow, [640, 480]).astype(int)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
)

cv2.imshow('mp feed', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# test angle
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

prev_frame_time = 0
new_frame_time = 0
frame_cnt = 0
total_fps = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity = 0) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        new_frame_time = time.time()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # print(results)
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        cv2.putText(image, 'FPS: {:.2f}'.format(fps), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

        cv2.imshow('mp feed', image)
        frame_cnt += 1
        total_fps += fps

        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Average FPS: ", total_fps/frame_cnt)
            break
cap.release()
cv2.destroyAllWindows()

Average FPS:  20.38380302333872
